In [1]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from tqdm import tqdm
import torch.nn.functional as F

2025-07-18 08:41:32.896507: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 08:41:32.904125: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752795692.912810 3261826 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752795692.915421 3261826 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 08:41:32.924979: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
CONFIG = {
    "data_base": "../data"
}

In [3]:
train_csv = pd.read_csv(f"{CONFIG['data_base']}/train.csv")
test_csv = pd.read_csv(f"{CONFIG['data_base']}/test.csv")

test_csv = test_csv.rename(columns={
    'paragraph_text': 'full_text'
})

train_csv.head()

,title,full_text,generated
0,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0
1,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0
2,엘자스-로트링겐 평의회 공화국,엘자스-로트링겐 평의회 공화국은 1차대전 말기 독일 혁명 와중에 엘자스-로트링겐에서...,0
3,윌리엄 페니 브룩스,"윌리엄 페니 브룩스(, 1809년 8월 13일 ~ 1895년 12월 11일)는 잉글...",0
4,미그로,"미그로 또는 미그로스(""Migros"")는 스위스 최대 소매 회사이자, 최대 슈퍼마켓...",0


In [4]:
train_df, val_df = train_test_split(
    train_csv,
    test_size=0.01,
    random_state=42,
    stratify=train_csv['generated']  # label 분포 유지
)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, data_df, tokenizer, mode='train'):
        self.data = data_df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        text = row['full_text']

        # 슬라이딩 윈도우 기반 tokenization
        inputs = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=512,
            stride=256,  # ✅ 겹치게 자름
            return_overflowing_tokens=True,
            return_tensors="pt"
        )

        # segment 중 하나 랜덤 선택
        n_segments = inputs["input_ids"].size(0)
        seg_idx = random.randint(0, n_segments - 1)

        item = {
            k: v[seg_idx] for k, v in inputs.items() if k != "overflow_to_sample_mapping"
        }

        # Longformer는 token_type_ids 없음
        item.pop("token_type_ids", None)

        if self.mode == 'train':
            item["labels"] = int(row["generated"])  # binary classification

        return item


In [6]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

train_dataset = CustomDataset(train_df, tokenizer, mode='train')
val_dataset = CustomDataset(val_df, tokenizer, mode='train')  # 선택적

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
base_model = AutoModelForSequenceClassification.from_pretrained(
    "klue/roberta-large",
    num_labels=2  # AI vs HUMAN → 이진 분류
)

base_model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],  # BERT 계열은 attention 부분 지정
    # target_modules=["query_proj", "value_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(base_model, lora_config)
model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(32000, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    auc = roc_auc_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'auroc': auc
    }

In [9]:
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")  # shape: (B,)
        outputs = model(**inputs)
        logits = outputs.logits  # shape: (B, C)

        # Label Smoothing
        smoothing = 0.0
        num_classes = logits.size(1)
        smoothed_labels = F.one_hot(labels, num_classes=num_classes).float()  # (B, C)
        smoothed_labels = smoothed_labels * (1 - smoothing) + smoothing / num_classes

        probs = torch.softmax(logits, dim=-1)
        ce_loss = - (smoothed_labels * torch.log(probs + 1e-8)).sum(dim=-1)  # shape: (B,)

        # 클래스별 weight 적용 (0: 1.0, 1: 12.0)
        class_weights = torch.tensor([1.0, 12.0], device=logits.device)
        example_weights = class_weights[labels]  # shape: (B,)

        loss = (ce_loss * example_weights).mean()

        return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir="../labeling_ckpt",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    max_steps=200000,
    learning_rate=5e-5,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=10000,
    logging_dir="../labeling_ckpt",
    logging_steps=10000,
    logging_strategy="steps",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    fp16=True,
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset, 
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/aicontest/anaconda3/envs/construct/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_3261826/2024928225.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss,Validation Loss


In [ ]:
test_dataset = CustomDataset(test_csv, tokenizer, mode='eval')

# 모델 준비
checkpoint_path = "../labeling_ckpt/checkpoint-200000"  # 원하는 checkpoint
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# 예측 저장 리스트
all_preds = []
all_probs = []

valid_keys = {"input_ids", "attention_mask"}

# ✅ 4. 추론 루프 (DataLoader 없이)
with torch.no_grad():
    for i in tqdm(range(len(test_dataset)), desc="Running inference without DataLoader"):
        batch = test_dataset[i]  # 단일 샘플 꺼내기
        # 모델이 받는 키만 선택
        inputs = {k: v.unsqueeze(0).to(model.device) for k, v in batch.items() if k in valid_keys}
        inputs.pop("token_type_ids", None)  # 혹시 있을 경우 제거

        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        preds = torch.argmax(probs, dim=-1)

        all_preds.extend(preds.cpu().tolist())
        all_probs.extend(probs.cpu().tolist())

sample_submission = pd.read_csv(f"{CONFIG['data_base']}/sample_submission.csv", encoding='utf-8-sig')
all_AI = [i[1] for i in all_probs]
sample_submission['generated'] = all_AI

sample_submission.to_csv(f"{CONFIG['data_base']}/baseline_submission.csv", index=False)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Running inference without DataLoader: 100%|██████████| 1962/1962 [00:51<00:00, 37.76it/s]
